In [23]:
#selector de columnas a desmantelar
def SSRC(layout):
    best_ev = 0; s_o=-1
    for i in range(len(layout.stacks)):
        #if layout.is_sorted_stack(i) and len(layout.stacks[i]) >= layout.H/3: continue
        if layout.is_sorted_stack(i) and Greedy.capacity(layout,i)<3*len(layout.stacks[i]): continue
        
        prom = sum(layout.stacks[i]) / len(layout.stacks[i]) 
        if layout.is_sorted_stack(i):
            ev = 100000 - 1000*len(layout.stacks[i]) - prom
        else:
            ev = 100000 - 1000*len(layout.stacks[i]) + prom
        

        if ev > best_ev:
            best_ev = ev
            s_o = i
    return s_o

In [43]:
def SDpp(layout, s_o):
    #Se calculan los ranks
    rank = Greedy.get_ranks(layout.stacks[s_o])
    
    capac = Greedy.capacity(layout,s_o) # espacio libre
    
    ss_o = layout.stacks[s_o]
    
    fixed_elements = 0

    #eliminar de rank elementos bien colocados
    while len(layout.stacks[s_o]) > 0: #fixed_elements:
        top = Layout.gvalue(ss_o)
        slack = capac-len(ss_o) # holgura
        #print(rank[top])
        s_d, xg = Layout.select_destination_stack (layout, s_o, max_pos=rank.index(top)+1+slack)#, rank=rank)
        #print(s_o, layout.stacks)
        pos = layout.H - len(layout.stacks[s_d])
        
        if rank.index(top)+1 < pos - slack:  # rellenar stack s_d
            #c=layout.stacks[s_o].pop(-1)
            #print()
            if verbose: 
                for s in layout.stacks: print(s)
                print("fill_stack")
                print(s_d,(pos - slack) - rank.index(top)-1,s_o, rank)
            success = Greedy.fill_stack(layout, s_d, (pos - slack) - (rank.index(top)+1), s_o, rank)
            if not success: return False, rank
            #layout.stacks[s_o].append(c)
        capac -= 1
        c = layout.move(s_o,s_d)
        if xg: rank.remove(c)
        if Layout.reachable_height(layout,s_o) >= layout.H: 
            for item in layout.stacks[s_o]: rank.remove(item)
            return True, rank
        
     #Se quitan los contenedores que se mantuvieron en su lugar para que no tire error la siguiente funcion
    for item in layout.stacks[s_o]: rank.remove(item)
    return True, rank

SFpp
---

Debería realizar un SF, priorizando los elementos (`rank`) que deberían ir en el stack original `s_d`.

````
while rank is not empty:
    s_o, s_dd = SF() # se obtienen los stacks asociados a un SF
    if s_d == s_dd: 
       if gvalue(s_dd) >= rank[0]: move(s_o, s_d)  #si stack coincide con original y el contenedor
                                                   #es mayor o igual al primero del ranking, se realiza el movimiento
       else: force_move(rank.pop(), s_d) #si no, se fuerza a mover siguiente contenedor del ranking (o mayor) a s_d
    else:
        move(s_o, s_dd)        #en caso de que SD no interfiera con el stack original, se realiza el movimiento y
        rank.pop(gvalue(s_dd))         #contenedor de elimina de stack
````



In [32]:
import sys
def SFpp(layout, s_d, rank):
    ub = Layout.gvalue(layout.stacks[s_d])
    conts = set()

    for r in rank:
        if r>ub: continue
        if r in conts: 
            conts.remove(r)
            continue
        
        while True:
            c = -1
            s, _ = Greedy.SF_move_(layout)
            s_o = None; s_dd = None
            if s != False: s_o, s_dd = s

            if s != False and s_d == s_dd and layout.stacks[s_o][-1] >= r: 
                c = layout.move(s_o,s_d)
                ub = c
            elif s==False or s_d == s_dd: #force move               
                if verbose==True:
                    if s!=False: print("rank:",r,"  c:",layout.stacks[s_o][-1], "s_o, s_d:",s_o,s_d)
                    else: print("rank:",r)
                    
                    print(layout.stacks)
                ret = Greedy.search_highest(layout, r, ub, s_d) 
                if ret == None: ub = r-1; break
                (s, pos) = ret

                c, state = Greedy.force_move(layout, s, pos, s_d) 
                if state == "failed" and ub == r: print ("NO DEBERIA"); break # NO DEBERIA OCURRIR
                ub = c
            else:
                c = layout.move(s_o, s_dd)

            if len(layout.stacks[s_d]) == layout.H: return True
            #if len(layout.stacks[s_d]) == layout.H-1 and layout.full_stacks >= len(layout.stacks)-3: return True
            
            if c==r: break
            conts.add(c)
    return True
        

In [4]:
def solve(layout, limit=1000):    
        Greedy.lazy_greedy(layout)

        while layout.unsorted_stacks>0 and layout.steps<limit:
            seleccionada = SSRC(layout)
        
            if verbose:
                for s in layout.stacks: print(s)
                print("\n")


            if verbose: 
                print("selected:",layout.stacks[seleccionada])
                #print("nb_unsorted_elements",nb_unsorted_elements)
                #print("ranks:",rank)

            #Se retiran los elementos de la columna
            success, rank = SDpp(layout, seleccionada)
            if not success: return 1000
            #gr.SD_move(layout, stack=seleccionada)

            if verbose:
                for s in layout.stacks: print(s)
                print("\n")


            #Según yo el último parámetro implica que el método tratará de llenar el stack, sin embargo si pongo 100 se cae?
            success = SFpp(layout,seleccionada, rank)#nb_unsorted_elements)
            if not success: return 1000

            Greedy.lazy_greedy(layout)


        #print("SELECCIONADA:", seleccionada)
        #print(len(layout.moves),"\t",layout.unsorted_stacks)
        return layout.steps
        #print()

In [5]:
import copy
def BSG(layout, w=4):
    best = 200
    S = [layout]
    while len(S)>0:
        C=[]
        for lay in S:
            C = C + generate_candidates(lay)
        SS = []
        #print(len(C))
        for clay in C:
            steps_old = clay.steps
            lb_steps = steps_old + (clay.total_elements-sum(clay.sorted_elements))
            gclay=copy.deepcopy(clay)
            steps = solve(gclay, limit=best+20)
            if steps < best and gclay.unsorted_stacks==0: 
                best=steps; 
                if bsg_verbose: 
                    print(best, gclay.stacks) 
                    print(gclay.bsg_moves)
            if steps < 1000 and steps_old < steps and lb_steps < steps:
                SS.append((steps,clay))
        SS.sort(key=lambda x: x[0])
        #print(SS)
        S.clear(); i=0
        for s in SS:
            if i==w: break
            i+=1
            if s[1].steps < best:
                S.append(s[1])     
        #print(len(S))
    return best

In [6]:
def generate_candidates(lay):
    steps = lay.steps
    C = []
    for i in range(len(lay.stacks)):
        clay = copy.deepcopy(lay)
        success, clay.rank = SDpp(clay, i)
        clay.last_sd = i
        #if success != True: continue
        #success = SFpp(clay,i, rank)
        if success and clay.steps > steps:
            clay.bsg_moves.append('SDpp'+str(i))
            C.append(clay)
            
    for i in range(len(lay.stacks)):
        clay = copy.deepcopy(lay)
        success, __ = Greedy.SD_move(clay, stack=i)
        clay.last_sd = None
        #if success != True: continue
        #success = SFpp(clay,i, rank)
        if success and clay.steps > steps:
            C.append(clay)
            
    clay = copy.deepcopy(lay) 
    clay.rank = None
    clay.last_sd = None
    Greedy.lazy_greedy(clay)
    if clay.steps > steps:
        clay.bsg_moves.append('LG')
        C.append(clay)
        
    clay = copy.deepcopy(lay) 
    if clay.last_sd != None:
        success = SFpp(clay, clay.last_sd, clay.rank)
        if success and clay.steps > steps:
            clay.bsg_moves.append('SFpp'+str(clay.last_sd))
            clay.last_sd=None
            clay.rank=None
            C.append(clay)
        

        
    return C

### Solve all the instances

In [7]:
import sys

def progressbar(it, prefix="", size=60, file=sys.stdout, out=['None']):
    count = len(it)
    def show(j):
        x = int(size*j/count)
        file.write("%s[%s%s] %s\r" % (prefix, "#"*x, "."*(size-x), out[0]))
        file.flush()        
    show(0)
    for i, item in enumerate(it):
        yield item
        show(i+1)
    #file.write("\n")
    file.write(prefix + out[0] + " "*2*size+"\n")
    file.flush()

import time

for i in progressbar(range(15), "Computing: ", 40):
    time.sleep(0.1) # any calculation you need

for i in progressbar(range(15), "Computing: ", 40):
    time.sleep(0.1) # any calculation you need

Computing: None                                                                                
Computing: None                                                                                


In [44]:
import os
import copy
import numpy as np
import Greedy as gr
import Greedy
import Pixie as px
import Layout
import time
import importlib
importlib.reload(gr)
importlib.reload(Layout)
importlib.reload(px)
from os import listdir
from os.path import isfile, join


onlyfiles = [f for f in listdir("Instancias/CVS") if not isfile(join("Instancias/CVS", f))]
verbose = False
bsg_verbose=False

tot_moves=0
for f in onlyfiles:
    if f == 'BF' or f== '3-x' : continue
    #print(f)
    nmoves=0; count = 0
    av = ["None"]
    start = time.time()
    for i in progressbar(range(1,41), "CVS-"+f+ ":\t", 40, out=av):
        filename = "Instancias/CVS/"+ f+ "/data"+f+"-"+str(i)+".dat"
        layout = px.read(filename)
        steps = solve(layout)
        #steps = BSG(layout, w=20)
        #print(filename, steps)
        if steps < 1000:
            count += 1
            nmoves += steps
            av[0] = str(nmoves/count)+"\t"+str(time.time() - start) +" "*10
        else: print(filename+"\n")
    tot_moves += nmoves
    #print (f,nmoves/ count,count, " "*50);
    sys.exit
print("prom CVS:",tot_moves/(21.0*40.0))

verbose=False
dir = "Instancias\\BF"
tot_moves=0
carpetas = os.listdir(dir)
#av = ["None"]
for i in range(1,33):
    carpeta = "BF"+str(i)
    archivos = os.listdir(dir+"\\"+carpeta)
    suma = 0
    count = 0
    start = time.time()
    for archivo in archivos:
        if "optimo" not in archivo: 
            ruta = dir+"\\"+carpeta+"\\"+archivo
            #print(ruta)
            layout = px.readBF(ruta)
            steps = solve(layout)
            #steps = BSG(layout, w=20)
            if steps < 1000:
                suma += steps
                count += 1
                #av[0] = str(nmoves/count)+" "+str(time.time() - start) +" "*10
            #suma += gr.greedy_solve(layout)
            
        
    promedio = suma/count  
    tot_moves += promedio
    print(carpeta + ":\t"+str(promedio)+"\t"+str(time.time() - start))
print("prom BF:",tot_moves/(32.0))

CVS-10-10:	163.575	0.3510596752166748                                                                                          
CVS-10-6:	126.7	0.17752385139465332                                                                                          
CVS-3-3:	11.775	0.07978653907775879                                                                                          
CVS-3-4:	11.875	0.07579708099365234                                                                                          
CVS-3-5:	12.575	0.07779169082641602                                                                                          
CVS-3-6:	13.375	0.0907599925994873                                                                                          
CVS-3-7:	14.975	0.0967402458190918                                                                                          
CVS-3-8:	15.5	0.10172843933105469                                                                                     

KeyboardInterrupt: 

In [124]:
importlib.reload(Layout)
verbose = False
bsg_verbose = True
filename = "Instancias/10-10/data10--1.dat"
print(layout.stacks)
layout = px.read(filename)

BSG(layout, w=4)
#layout.stacks, layout.full_stacks

[[95, 90, 88, 86, 83, 79, 72, 68, 64, 46, 43], [78, 25, 48, 81, 59, 23, 47, 89, 4, 53], [1, 24, 77, 35, 69, 42, 66, 17, 16, 18, 19, 27], [60, 37], [61, 41, 58, 50, 67, 80, 39, 28, 45, 33, 55, 94], [65, 63, 51, 26, 7], [15, 87, 62, 97, 93, 5, 98, 34, 36, 57, 85, 92], [30, 12, 11, 8, 9, 22, 10, 91, 20, 29, 75, 76], [2, 84, 96, 56, 71, 49, 13, 21, 52, 6, 31, 32], [82, 44, 74, 40, 73, 100, 54, 99, 38, 70, 3, 14]]
160 [[95, 90, 88, 86, 83, 79, 72, 68, 64, 46, 43, 37], [78, 59, 48, 47, 33, 31, 25, 23, 20, 16, 14, 6], [89, 81, 77, 69, 66, 52, 45, 42, 36, 35, 34, 32], [76, 75, 60, 53], [80, 67, 61, 58, 50, 41, 39, 15, 13, 10, 5, 4], [92, 85, 70, 65, 63, 57, 55, 38, 27, 26, 7], [98, 97, 93, 91, 87, 62, 29, 28, 24, 21, 18, 17], [30, 12, 11, 8, 3, 2, 1], [96, 84, 71, 56, 49, 22, 19, 9], [100, 99, 94, 82, 74, 73, 54, 51, 44, 40]]
['SDpp0']
159 [[95, 90, 89, 88, 86, 83, 81, 79, 72, 70, 46, 45], [78, 68, 64, 59, 48, 47, 33, 32, 29, 25, 23, 20], [77, 69, 35, 24, 22], [76, 75, 60, 53, 26, 19, 7, 2, 1]

153

In [39]:
importlib.reload(Layout)
verbose = True
filename = "Instancias/CVS/10-6/data10-6-9.dat"
layout = px.read(filename)
solve(layout)
#for s in layout.stacks: print(s)

[8, 43, 25, 48, 42, 17, 54, 51, 35, 45]
[6, 14, 31, 9, 33, 18, 20, 1, 34, 30]
[13, 44, 4, 15, 39, 28, 7, 40, 52, 36]
[3, 26, 2, 23, 59, 12, 11, 47, 46, 49]
[10, 32, 21, 19, 50, 41, 56, 58, 27, 53]
[22, 29, 60, 24, 57, 37, 38, 55, 5, 16]


selected: [8, 43, 25, 48, 42, 17, 54, 51, 35, 45]
[8, 43, 25, 48, 42, 17, 54]
[6, 14, 31, 9, 33, 18, 20, 1, 34, 30, 35]
[13, 44, 4, 15, 39, 28, 7, 40, 52, 36, 45]
[3, 26, 2, 23, 59, 12, 11, 47, 46, 49, 51]
[10, 32, 21, 19, 50, 41, 56, 58, 27, 53]
[22, 29, 60, 24, 57, 37, 38, 55, 5, 16]
fill_stack
4 1 0 [54, 51, 48, 45, 43, 42, 35, 25, 17, 8]
[]
[6, 14, 31, 9, 33, 18, 20, 1, 34, 30, 35, 42]
[13, 44, 4, 15, 39, 28, 7, 40, 52, 36, 45, 48]
[3, 26, 2, 23, 59, 12, 11, 47, 46, 49, 51, 8]
[10, 32, 21, 19, 50, 41, 56, 58, 27, 53, 16, 54]
[22, 29, 60, 24, 57, 37, 38, 55, 5, 17, 25, 43]


rank: 51   c: 48 s_o, s_d: 2 0
[[54], [6, 14, 31, 9, 33, 18, 20, 1, 34, 30, 35, 42], [13, 44, 4, 15, 39, 28, 7, 40, 52, 36, 45, 48], [3, 26, 2, 23, 59, 12, 11, 47, 46, 49, 51, 

139

In [118]:
success, ranks = SDpp(layout, 0)
print(success)
for s in layout.stacks: print(s)

True
[]
[78, 25, 48, 81, 59, 23, 47, 89, 83, 88, 90]
[1, 24, 77, 35, 69, 42, 66, 17, 16, 72, 79]
[60, 7, 53, 26, 76, 75, 51, 94, 92, 86]
[61, 41, 58, 50, 67, 80, 39, 28, 45, 33, 43, 46]
[65, 63, 85, 55, 57, 27, 19, 4, 18, 29, 32, 37]
[15, 87, 62, 97, 93, 5, 98, 34, 36, 95]
[30, 12, 11, 8, 9, 22, 10, 91, 20, 64, 68]
[2, 84, 96, 56, 71, 49, 13, 21, 52, 6, 31]
[82, 44, 74, 40, 73, 100, 54, 99, 38, 70, 3, 14]


In [119]:
success = SFpp(layout, 0, ranks)
print(success)
for s in layout.stacks: print(s)

True
[95, 90, 88, 86, 83, 79, 72, 68, 64, 46, 43, 37]
[78, 25, 48, 81, 59, 23, 47, 89]
[1, 24, 77, 35, 69, 42, 66, 17, 16]
[60, 7, 53, 26, 76, 75, 51, 94, 92]
[61, 41, 58, 50, 67, 80, 39, 28, 45, 33]
[65, 63, 85, 55, 57, 27, 19, 4, 18, 29, 32]
[15, 87, 62, 97, 93, 5, 98, 34, 36]
[30, 12, 11, 8, 9, 22, 10, 91, 20]
[2, 84, 96, 56, 71, 49, 13, 21, 52, 6, 31]
[82, 44, 74, 40, 73, 100, 54, 99, 38, 70, 3, 14]


In [120]:
success, ranks = SDpp(layout, 5)
print(success)
for s in layout.stacks: print(s)

True
[95, 90, 88, 86, 83, 79, 72, 68, 64, 46, 43, 37]
[78, 25, 48, 81, 59, 23, 47, 89, 4]
[1, 24, 77, 35, 69, 42, 66, 17, 16, 18, 19, 27]
[60, 7, 53, 26, 76, 75, 51, 94, 92]
[61, 41, 58, 50, 67, 80, 39, 28, 45, 33, 55]
[65, 63]
[15, 87, 62, 97, 93, 5, 98, 34, 36, 57, 85]
[30, 12, 11, 8, 9, 22, 10, 91, 20, 29]
[2, 84, 96, 56, 71, 49, 13, 21, 52, 6, 31, 32]
[82, 44, 74, 40, 73, 100, 54, 99, 38, 70, 3, 14]


In [121]:
success, ranks = SDpp(layout, 3)
print(success)
for s in layout.stacks: print(s)

True
[95, 90, 88, 86, 83, 79, 72, 68, 64, 46, 43, 37]
[78, 25, 48, 81, 59, 23, 47, 89, 4, 53]
[1, 24, 77, 35, 69, 42, 66, 17, 16, 18, 19, 27]
[60]
[61, 41, 58, 50, 67, 80, 39, 28, 45, 33, 55, 94]
[65, 63, 51, 26, 7]
[15, 87, 62, 97, 93, 5, 98, 34, 36, 57, 85, 92]
[30, 12, 11, 8, 9, 22, 10, 91, 20, 29, 75, 76]
[2, 84, 96, 56, 71, 49, 13, 21, 52, 6, 31, 32]
[82, 44, 74, 40, 73, 100, 54, 99, 38, 70, 3, 14]


In [122]:
success, ranks = SDpp(layout, 0)
print(success)
for s in layout.stacks: print(s)

True
[95, 90, 88, 86, 83, 79, 72, 68, 64, 46, 43]
[78, 25, 48, 81, 59, 23, 47, 89, 4, 53]
[1, 24, 77, 35, 69, 42, 66, 17, 16, 18, 19, 27]
[60, 37]
[61, 41, 58, 50, 67, 80, 39, 28, 45, 33, 55, 94]
[65, 63, 51, 26, 7]
[15, 87, 62, 97, 93, 5, 98, 34, 36, 57, 85, 92]
[30, 12, 11, 8, 9, 22, 10, 91, 20, 29, 75, 76]
[2, 84, 96, 56, 71, 49, 13, 21, 52, 6, 31, 32]
[82, 44, 74, 40, 73, 100, 54, 99, 38, 70, 3, 14]


![image](https://i.imgur.com/XsBW9YR.png)

![image](https://i.imgur.com/uE2o976.png)

In [258]:
verbose = False 
filename = "Instancias/3-3/data3-3-38.dat"
layout = px.read(filename)
print (layout.stacks)

[[3, 6, 8], [7, 9, 5], [2, 1, 4]]


In [184]:
C=generate_candidates(layout)

In [25]:
importlib.reload(Layout)

total=0
for i in range(1,2):
    verbose = False 
    filename = "Instancias\BF\BF5\cpmp_16_5_64_13_39_4.bay"
    layout = px.readBF(filename)
    steps=BSG(layout, w=10)
    total+=steps
    print(filename,steps)
print(total/40.0)

Instancias\BF\BF5\cpmp_16_5_64_13_39_4.bay 41
1.025


In [191]:
C[1].unsorted_stacks

0

In [192]:
solve(C[1])

14